In [ ]:
from wrf.extension import _slp

In [ ]:
from wrf import getvar
from netCDF4 import Dataset as nc
ncfile = nc("/Users/ladwig/Documents/wrf_files/wrf_vortex_multi/wrfout_d01_2005-08-28_00:00:00")
b = getvar([ncfile,ncfile,ncfile], "slp", timeidx=None)

In [ ]:
print(b)

In [ ]:
b = getvar([ncfile,ncfile,ncfile], "td", None)

In [ ]:
print(b)


In [ ]:
b = getvar([ncfile,ncfile,ncfile], "tk", None)

In [ ]:
print(b)

In [ ]:
b = getvar([ncfile,ncfile,ncfile], "rh", None)

In [ ]:
print (b)

In [ ]:
# 500 MB Heights
from wrf import getvar, interplevel

z = getvar([ncfile,ncfile,ncfile], "z", timeidx=None)
p = getvar([ncfile,ncfile,ncfile], "pressure", timeidx=None)
ht_500mb = interplevel(z, p, 500)

print(ht_500mb)
del ht_500mb, z, p

In [ ]:
# Pressure using pivot and angle
from wrf import getvar, vertcross

z = getvar(ncfile, "z", timeidx=None)
p = getvar(ncfile, "pressure", timeidx=None)
pivot_point = (z.shape[-2] / 2, z.shape[-1] / 2) 
angle = 90.0

p_vert = vertcross(p, z, pivot_point=pivot_point, angle=angle)
print(p_vert)
del p_vert

# Pressure using start_point and end_point
start_point = (z.shape[-2]/2, 0)
end_point = (z.shape[-2]/2, -1)

p_vert = vertcross(p, z, start_point=start_point, end_point=end_point)
print(p_vert)
del p_vert, p, z


In [ ]:
# T2 using pivot and angle
from wrf import interpline, getvar

t2 = getvar([ncfile,ncfile,ncfile], "T2", timeidx=None)
pivot_point = (t2.shape[-2] / 2, t2.shape[-1] / 2) 
angle = 90.0

t2_line = interpline(t2, pivot_point=pivot_point, angle=angle)
print(t2_line)

del t2_line

# T2 using start_point and end_point
start_point = (t2.shape[-2]/2, 0)
end_point = (t2.shape[-2]/2, -1)

t2_line = interpline(t2, start_point=start_point, end_point=end_point)
print(t2_line)

del t2_line, t2

In [ ]:
from wrf import getvar
from netCDF4 import Dataset as nc
lambertnc = nc("/Users/ladwig/Documents/wrf_files/wrfout_d01_2010-06-13_21:00:00")
uvmet = getvar([lambertnc,lambertnc], "uvmet", timeidx=None)
uvmet10 = getvar([lambertnc,lambertnc], "uvmet10", timeidx=None)
uv_wspd_wdir = getvar([lambertnc,lambertnc], "wspd_wdir_uvmet", timeidx=None)
uv_wspd_wdir10 = getvar([lambertnc,lambertnc], "wspd_wdir_uvmet10", timeidx=None)
wspd_wdir = getvar([lambertnc,lambertnc], "wspd_wdir", timeidx=None)
wspd_wdir10 = getvar([lambertnc,lambertnc], "wspd_wdir10", timeidx=None)
print (uvmet)
print (uvmet10)
print (uv_wspd_wdir)
print (uv_wspd_wdir10)
print (wspd_wdir)
print (wspd_wdir10)

In [ ]:
from wrf import (ALL_TIMES, to_np, Constants, getvar, extract_vars, destagger, 
                 interp1d, interp2dxy, interpz3d, 
                 slp, tk, td, rh, uvmet, smooth2d)
from netCDF4 import Dataset as nc

wrfnc = nc("/Users/ladwig/Documents/wrf_files/wrf_vortex_multi/wrfout_d01_2005-08-28_00:00:00")
timeidx = ALL_TIMES
method = "cat"
squeeze = True
cache = None



In [ ]:

varnames=("T", "P", "PB", "QVAPOR", "PH", "PHB")
ncvars = extract_vars(wrfnc, timeidx, varnames, method, squeeze, cache,
                          meta=True)

t = ncvars["T"]
p = ncvars["P"]
pb = ncvars["PB"]
ph = ncvars["PH"]
phb = ncvars["PHB"]


ncvars = extract_vars(wrfnc, timeidx, ("QVAPOR",), method, squeeze, cache,
                          meta=False)
qvapor = ncvars["QVAPOR"]

    
full_t = t + Constants.T_BASE
full_p = p + pb
qvapor[qvapor < 0] = 0.
    
full_ph = (ph + phb) / Constants.G
    
destag_ph = destagger(full_ph, -3)
    
_tk = tk(full_p, full_t)
_slp = slp(destag_ph, _tk, full_p, qvapor)
_td = td(full_p, qvapor)
_smooth2d = smooth2d(to_np(_slp), 3)

_uvmet = getvar(wrfnc, "uvmet", timeidx=1)

In [ ]:
print (_tk)
print ("\n")
print (_slp)
print ("\n")
print (_td)
print ("\n")
print (_smooth2d)
print (_uvmet)

In [ ]:
from wrf import (ALL_TIMES, to_np, Constants, getvar, extract_vars, destagger, 
                 interp1d, interp2dxy, interpz3d, 
                 slp, tk, td, rh, uvmet, smooth2d, extract_global_attrs)
from math import fabs, log, tan, sin, cos
from wrf.util import either
from netCDF4 import Dataset as nc

wrfnc = nc("/Users/ladwig/Documents/wrf_files/wrfout_d01_2010-06-13_21:00:00")
wrfnc = [wrfnc, wrfnc, wrfnc]
ten_m = True
method = "cat"
squeeze=True
cache=None
timeidx = ALL_TIMES

if not ten_m:
    varname = either("U", "UU")(wrfnc)
    u_vars = extract_vars(wrfnc, timeidx, varname, method, squeeze, cache,
                          meta=True)
    
    #renamed = u_vars[varname].rename({"west_east_stag" : "test"})
    u = destagger(u_vars[varname], -1, meta=True)
    #print (renamed)
    #u = destagger(renamed, -1, meta=True)
    #print (u)

    varname = either("V", "VV")(wrfnc)
    v_vars = extract_vars(wrfnc, timeidx, varname, method, squeeze, cache,
                          meta=True)
    v = destagger(v_vars[varname], -2)
else:
    varname = either("U10", "UU")(wrfnc)
    u_vars = extract_vars(wrfnc, timeidx, varname, method, squeeze, cache,
                          meta=True)
    u = (u_vars[varname] if varname == "U10" else 
         destagger(u_vars[varname][...,0,:,:], -1)) 

    varname = either("V10", "VV")(wrfnc)
    v_vars = extract_vars(wrfnc, timeidx, varname, method, squeeze, cache,
                          meta=True)
    v = (v_vars[varname] if varname == "V10" else 
         destagger(v_vars[varname][...,0,:,:], -2))

map_proj_attrs = extract_global_attrs(wrfnc, attrs="MAP_PROJ")
map_proj = map_proj_attrs["MAP_PROJ"]

# 1 - Lambert
# 2 - Polar Stereographic
# 3 - Mercator
# 6 - Lat/Lon
# Note:  NCL has no code to handle other projections (0,4,5) as they 
# don't appear to be supported any longer

if map_proj in (0,3,6):
    # No rotation needed for Mercator and Lat/Lon, but still need
    # u,v aggregated in to one array

    end_idx = -3 if not ten_m else -2
    resdim = (2,) + u.shape[0:end_idx] + u.shape[end_idx:]

    # Make a new output array for the result
    res = np.empty(resdim, u.dtype)

    # For 2D array, this makes (0,...,:,:) and (1,...,:,:)
    # For 3D array, this makes (0,...,:,:,:) and (1,...,:,:,:)
    idx0 = (0,) + (Ellipsis,) + (slice(None),)*(-end_idx)
    idx1 = (1,) + (Ellipsis,) + (slice(None),)*(-end_idx)

    res[idx0] = u[:]
    res[idx1] = v[:]

    result = res
elif map_proj in (1,2):
    lat_attrs = extract_global_attrs(wrfnc, attrs=("TRUELAT1",
                                                   "TRUELAT2"))
    radians_per_degree = Constants.PI/180.0
    # Rotation needed for Lambert and Polar Stereographic
    true_lat1 = lat_attrs["TRUELAT1"]
    true_lat2 = lat_attrs["TRUELAT2"]

    try:
        lon_attrs = extract_global_attrs(wrfnc, attrs="STAND_LON")
    except AttributeError:
        try:
            cen_lon_attrs = extract_global_attrs(wrfnc, attrs="CEN_LON")
        except AttributeError:
            raise RuntimeError("longitude attributes not found in NetCDF")
        else:
            cen_lon = cen_lon_attrs["CEN_LON"]
    else:
        cen_lon = lon_attrs["STAND_LON"]


    varname = either("XLAT_M", "XLAT")(wrfnc)
    xlat_var = extract_vars(wrfnc, timeidx, varname, 
                            method, squeeze, cache, meta=True)
    lat = xlat_var[varname]

    varname = either("XLONG_M", "XLONG")(wrfnc)
    xlon_var = extract_vars(wrfnc, timeidx, varname, 
                            method, squeeze, cache, meta=False)
    lon = xlon_var[varname]

    if map_proj == 1:
        if((fabs(true_lat1 - true_lat2) > 0.1) and
                (fabs(true_lat2 - 90.) > 0.1)): 
            cone = (log(cos(true_lat1*radians_per_degree)) 
                - log(cos(true_lat2*radians_per_degree)))
            cone = (cone / 
                    (log(tan((45.-fabs(true_lat1/2.))*radians_per_degree)) 
                - log(tan((45.-fabs(true_lat2/2.))*radians_per_degree)))) 
        else:
            cone = sin(fabs(true_lat1)*radians_per_degree)
    else:
        cone = 1
    
    result = uvmet(u, v, lat, lon, cen_lon, cone, meta=True)
    
print (result)

In [ ]:
import numpy as np
from wrf import (ALL_TIMES, to_np, Constants, getvar, extract_vars, destagger, 
                 interp1d, interp2dxy, interpz3d, 
                 slp, tk, td, rh, uvmet, smooth2d, extract_global_attrs, xy)
from math import fabs, log, tan, sin, cos
from wrf.util import either
from netCDF4 import Dataset as nc

timeidx = None
wrfnc = nc("/Users/ladwig/Documents/wrf_files/wrfout_d01_2010-06-13_21:00:00")

wrfnc = [wrfnc, wrfnc]
field3d = getvar(wrfnc, "P", timeidx, meta=True)

z = getvar(wrfnc, "z", timeidx)

interp = interpz3d(field3d, z, 500, missingval=Constants.DEFAULT_FILL, meta=True)
print (interp)
print ("\n")

# 2dxy test
pivot_point = (field3d.shape[-1] / 2, field3d.shape[-2] / 2 ) 
angle = 90.0

start_point = (0, z.shape[-2]/2)
end_point = (-1, z.shape[-2]/2)

#start_point = (0, 0)
#end_point = (-1, -1)

_xy = xy(field3d, pivot_point=pivot_point, angle=angle)
print(_xy)
print ("\n")

_xy = xy(field3d, start_point=start_point, end_point=end_point)
print(_xy)
print ("\n")

interpxy = interp2dxy(field3d, _xy)
print (interpxy)
print ("\n")

# 1D test
only_height = [0]*field3d.ndim
only_height[-3] = slice(None)
only_height = (Ellipsis,) + tuple(only_height[-3:])

v_in = field3d[only_height]
print("v_in", v_in)
z_in = z[only_height]
z_out = np.asarray([100.,200.,300.,500.,1000.,5000.], field3d.dtype)

int1d = interp1d(v_in, z_in, z_out, missingval=Constants.DEFAULT_FILL, meta=True)
print(int1d)
print ("\n")


In [ ]:

import numpy as np

from wrf import (ALL_TIMES, getvar, cape_2d, cape_3d, extract_vars, tk, destagger, ConversionFactors, Constants)
from netCDF4 import Dataset as nc

timeidx = ALL_TIMES
method = "cat"
squeeze=True
cache=None
timeidx = None

wrfnc = nc("/Users/ladwig/Documents/wrf_files/wrfout_d01_2010-06-13_21:00:00")

wrfnc = [wrfnc, wrfnc]

varnames = ("T", "P", "PB", "QVAPOR", "PH", "PHB", "HGT", "PSFC")
ncvars = extract_vars(wrfnc, timeidx, varnames, method, squeeze, cache,
                          meta=True)
t = ncvars["T"]
p = ncvars["P"]
pb = ncvars["PB"]
qvapor = ncvars["QVAPOR"]
ph = ncvars["PH"]
phb = ncvars["PHB"]
ter = ncvars["HGT"]
psfc = ncvars["PSFC"]
    
full_t = t + Constants.T_BASE
full_p = p + pb
tkel = tk(full_p, full_t)
    
geopt = ph + phb
geopt_unstag = destagger(geopt, -3)
height = geopt_unstag/Constants.G
    
# Convert pressure to hPa
p_hpa = ConversionFactors.PA_TO_HPA * full_p
psfc_hpa = ConversionFactors.PA_TO_HPA * psfc 

ter_follow = 1

cape2d = cape_2d(p_hpa, tkel, qvapor, height, ter, psfc_hpa, ter_follow, 
            missing=Constants.DEFAULT_FILL, meta=True)

cape3d = cape_3d(p_hpa, tkel, qvapor, height, ter, psfc_hpa, ter_follow, 
            missing=Constants.DEFAULT_FILL, meta=True)

print (cape2d)
print (cape3d)

